In [0]:
#!pip install psycopg2-binary

In [7]:
!pip install category_encoders

     |████████████████████████████████| 92kB 3.6MB/s 


In [8]:
!git clone https://github.com/labs15-baseball-pitch-predictor/RC-v1.1--Data-Cleaning-and-Feature-Engineering.git

Cloning into 'RC-v1.1--Data-Cleaning-and-Feature-Engineering'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 123 (delta 9), reused 0 (delta 0), pack-reused 107
Receiving objects: 100% (123/123), 277.04 MiB | 51.04 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [0]:
import pandas as pd
import numpy as np
import random
import pickle
import gc
import psycopg2
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Load in the pre-pitcher preprocessed_df



In [0]:
#load in pickled preprocessed df

df1 = pd.read_pickle("/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/pre_pitcher_pre_processed_df_part1.pkl", compression='zip')
df2 = pd.read_pickle("/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/pre_pitcher_pre_processed_df_part2.pkl", compression='zip')

df = pd.concat([df1, df2])

In [11]:
#clear some memory:
del df1
del df2
gc.collect()

0

In [0]:
#make list of pitchers (for filling in Nans in batting stat later for pitcher slot in batting order)

def make_pitcher_list(pre_processed_df):
    #make list of all the pitcher ids in the preprocessed_df
    pitcher_list = df['pitcher'].unique().tolist()
    
    #on rare occassions in extra-inning games, a position player is called in to pitch: 
    #to avoid categorizing him as a pitcher we'll only count pitchers with over 100 pitches:
    for pitcher in pitcher_list:
        if len(df[df['pitcher'] == pitcher]) < 100:
            pitcher_list.remove(pitcher)
    return pitcher_list

pitcher_list = make_pitcher_list(df)

## Choose the pitcher to model

In [0]:
#df.groupby('pitcher').count().sort_values('pitch_type', ascending=False).head(10)

In [14]:
#dictionary mapping top 10 pitchers with most pitches name with pitcher id

pitcher_id_dict = {'Justin Verlander': 434378.0, 'Trevor Bauer': 545333.0, 
                   'Aaron Nola': 605400.0, 'Lance Lynn': 458681.0, 'Gerrit Cole':
                   543037.0, 'Jacob deGrom':594798.0, 'Zack Greinke':425844.0,
                   'Patrick Corbin': 571578.0, 'Max Scherzer': 453286.0,
                   'Kyle Gibson': 502043.0}

#make pitcher_df subset where pitcher == picther_id
pitcher_id = pitcher_id_dict['Justin Verlander']
# pitcher_id = pitcher_id_dict['Trevor Bauer']
# pitcher_id = pitcher_id_dict['Aaron Nola']
# pitcher_id = pitcher_id_dict['Lance Lynn']
# pitcher_id = pitcher_id_dict['Gerrit Cole']
# pitcher_id = pitcher_id_dict['Jacob deGrom']
# pitcher_id = pitcher_id_dict['Zack Greinke']
# pitcher_id = pitcher_id_dict['Patrick Corbin']
# pitcher_id = pitcher_id_dict['Max Scherzer']
# pitcher_id = pitcher_id_dict['Kyle Gibson']

pitcher_df = df[df['pitcher'] == pitcher_id]
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat
1510,FF,2018-03-29,22730,94.599998,-1.9486,6.8034,592261.0,434378.0,8.0,NaN,2018.0,-0.8339,1.6529,0.2748,1.7070,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,7.4453,-136.895203,-10.8637,-11.885600,32.316799,-9.809500,3.3339,1.4562,NaN,NaN,NaN,92.489998,2652.0,5.498,529411.0,55.000801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,S,NaN,Bot,180329_195119,Standard,1.0,NaN,False,False,False,False,0,0,1,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,0,Justin Verlander,NaN,called_strike,NaN,4-Seam Fastball,Standard,00,neutral,fastball
1511,FF,2018-03-29,22711,94.300003,-1.9916,6.6522,592261.0,434378.0,14.0,NaN,2018.0,-1.2892,1.4338,0.4047,1.2020,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.9916,-136.566605,-11.4425,-18.015499,31.964199,-12.041500,3.3446,1.4668,NaN,NaN,NaN,93.283997,2638.0,6.095,529411.0,54.403400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195137,Standard,1.0,NaN,False,True,False,False,0,0,0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,1,Justin Verlander,NaN,ball,NaN,4-Seam Fastball,Standard,01,ahead,fastball
1512,SL,2018-03-29,22699,86.400002,-2.1580,6.5734,592261.0,434378.0,14.0,NaN,2018.0,0.2707,0.3526,1.5138,-0.0429,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.0583,-125.145203,-10.0136,1.186800,24.599800,-26.845699,3.3339,1.4562,NaN,NaN,NaN,84.944000,2680.0,5.543,529411.0,54.956001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195201,Standard,1.0,NaN,False,True,False,True,0,0,0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,1,1,Justin Verlander,NaN,ball,NaN,Slider,Standard,11,ahead,breaking
1513,FF,2018-03-29,22691,95.300003,-1.9823,6.7639,592261.0,434378.0,5.0,6.0,2018.0,-1.0804,1.4159,0.1783,2.3724,0.0,0.0,0.0,0.0,1.0,108.040001,149.179993,435263.0,7.9219,-138.185593,-8.8461,-15.272600,30.025200,-12.733900,3.1947,1.2860,145.0,104.199997,7.0,94.040001,2579.0,5.690,529411.0,54.808498,0.674,0.644,0.0,1.0,0.0,0.0,4.0,6.0,4.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,X,line_drive,Bot,180329_195226,Standard,1.0,NaN,False,False,False,False,0,1,1,0,66.241684,21.138212,42.050209,

## Add the pitcher prior tendencies (Pitcher Scouting Report)

In [15]:
%%time
#read in 2017 pre_procesed step 1 pickle and filter for pitcher_id
df17 = pd.read_pickle('/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/preprocessed_2017.pkl', compression = 'zip')
pitcher_df17 = df17[df17['pitcher'] == pitcher_id]

CPU times: user 2.25 s, sys: 131 ms, total: 2.38 s
Wall time: 2.39 s


In [0]:
def get_left_right_pitch_tendencies(pitcher_df):
    #split the df into left hand and right handed batters
    left = pitcher_df[pitcher_df['stand'] == 'L'].copy()
    right = pitcher_df[pitcher_df['stand'] == 'R'].copy()
    
    #assign the normalized value counts for this pitchers pitch types to a dictionary
    overall_left = left['pitch_cat'].value_counts(normalize=True).to_dict()
    overall_right = right['pitch_cat'].value_counts(normalize=True).to_dict()
    
    #initialize empty dict for count categories tendencies
    by_count_left = {}
    by_count_right = {}
    
    #loop over each count category and get the pitchers tendencies and add to the dict
    for cat in pitcher_df['count_cat'].unique().tolist():
        left_subset = left[left['count_cat'] == cat]
        right_subset = right[right['count_cat'] == cat]
        
        by_count_left[cat] = left_subset['pitch_cat'].value_counts(normalize=True).to_dict()
        by_count_right[cat] = right_subset['pitch_cat'].value_counts(normalize=True).to_dict()
        
    return overall_left, overall_right, by_count_left, by_count_right

def make_tendency_features(pitcher_df, overall_left, overall_right, by_count_left, by_count_right):
    #helper functions to vectorize w/ df.apply():
    def get_overall_left_perc(x):
        return overall_left[x] * 100
    def get_overall_right_perc(x):
        return overall_right[x] * 100
    def get_by_count_left_perc(x):
        try:
            return by_count_left[x][pitch_type] * 100
        except KeyError:
            return 0
    def get_by_count_right_perc(x):
        try:
            return by_count_right[x][pitch_type] * 100
        except KeyError:
            return 0
    
    left = pitcher_df[pitcher_df['stand'] == 'L'].copy()
    right = pitcher_df[pitcher_df['stand'] == 'R'].copy()
    
    pitch_types_left = overall_left.keys()
    pitch_types_right = overall_right.keys()
    
    #Left
    for pitch_type in pitch_types_left:
        overall_feature = 'overall_' + pitch_type + '_perc'
        count_cat_feature = 'count_cat_' + pitch_type + '_perc'
      
        left[overall_feature] = pitch_type
        left[overall_feature] = left[overall_feature].apply(get_overall_left_perc)
        left[count_cat_feature] = left['count_cat'].apply(get_by_count_left_perc)
    #Right
    for pitch_type in pitch_types_right:
        overall_feature = 'overall_' + pitch_type + '_perc'
        count_cat_feature = 'count_cat_' + pitch_type + '_perc'
      
        right[overall_feature] = pitch_type
        right[overall_feature] = right[overall_feature].apply(get_overall_right_perc)
        right[count_cat_feature] = right['count_cat'].apply(get_by_count_right_perc)
        
    
    return pd.concat([left,right], sort=False).sort_values(by=['game_date', 'game_pk', 'at_bat_number', 'pitch_number'])

def add_pitcher_scouting_report(pitcher_df, pitcher_df17, start_dates, end_dates):
    df = pd.concat([pitcher_df, pitcher_df17], sort=False)
    
    #initialize empty list to store dfs (concat them together later)
    df_list = []
    
    #iterate over each period
    for i in range(len(start_dates)):
      
        #make the prior and current dfs:
        prior_df = df[df['game_date'] < start_dates[i]]
        current_df = df[(df['game_date'] >= start_dates[i]) & (df['game_date'] <= end_dates[i])].copy()
        
        #get the pitch tendencies from prior:
        overall_left, overall_right, by_count_left, by_count_right = get_left_right_pitch_tendencies(prior_df)
        
        #make the pitch tendencies features on current:
        current_df = make_tendency_features(current_df, overall_left, overall_right, by_count_left, by_count_right)
        
        #append the df to the list
        df_list.append(current_df)
    
    df = pd.concat(df_list, sort=False)
    return df


In [0]:
start_dates = ['2018-03-29', '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01', 
               '2018-09-01', '2019-03-28', '2019-05-01', '2019-06-01', '2019-07-01', 
               '2019-08-01']

end_dates =  ['2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', 
              '2018-10-01', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', 
              '2019-08-31']

In [18]:
%%time
pitcher_df = add_pitcher_scouting_report(pitcher_df, pitcher_df17, start_dates, end_dates)

CPU times: user 1.23 s, sys: 44.7 ms, total: 1.28 s
Wall time: 1.29 s


In [19]:
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_fastball_perc,count_cat_fastball_perc,overall_breaking_perc,count_cat_breaking_perc,overall_offspeed_perc,count_cat_offspeed_perc
1510,FF,2018-03-29,22730,94.599998,-1.9486,6.8034,592261.0,434378.0,8.0,NaN,2018.0,-0.8339,1.6529,0.2748,1.7070,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,7.4453,-136.895203,-10.8637,-11.885600,32.316799,-9.809500,3.3339,1.4562,NaN,NaN,NaN,92.489998,2652.0,5.498,529411.0,55.000801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,S,NaN,Bot,180329_195119,Standard,1.0,NaN,False,False,False,False,0,0,1,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,0,Justin Verlander,NaN,called_strike,NaN,4-Seam Fastball,Standard,00,neutral,fastball,59.546127,69.760900,38.973853,30.098453,1.48002,0.140647
1511,FF,2018-03-29,22711,94.300003,-1.9916,6.6522,592261.0,434378.0,14.0,NaN,2018.0,-1.2892,1.4338,0.4047,1.2020,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.9916,-136.566605,-11.4425,-18.015499,31.964199,-12.041500,3.3446,1.4668,NaN,NaN,NaN,93.283997,2638.0,6.095,529411.0,54.403400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195137,Standard,1.0,NaN,False,True,False,False,0,0,0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,1,Justin Verlander,NaN,ball,NaN,4-Seam Fastball,Standard,01,ahead,fastball,59.546127,50.609185,38.973853,47.141518,1.48002,2.249297
1512,SL,2018-03-29,22699,86.400002,-2.1580,6.5734,592261.0,434378.0,14.0,NaN,2018.0,0.2707,0.3526,1.5138,-0.0429,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.0583,-125.145203,-10.0136,1.186800,24.599800,-26.845699,3.3339,1.4562,NaN,NaN,NaN,84.944000,2680.0,5.543,529411.0,54.956001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195201,Standard,1.0,NaN,False,True,False,True,0,0,0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,1,1,Justin Verlander,NaN,ball,NaN,Slider,Standard,11,ahead,breaking,59.546127,50.609185,38.973853,47.141518,1.48002,2.249297
1513,FF,2018-03-29,22691,95.300003,-1.9823,6.7639,592261.0,434378.0,5.0,6.0,2018.0,-1.0804,1.4159,0.1783,2.3724,0.0,0.0,0.0,0.0,1.0,108.040001,149.179993,435263.0,7.9219,-138.185593,-8

## Add Game Features:

#### Batting Order, Game pitch_count, & Trailing Pitch Features

In [0]:
def make_game_batting_order(game_df):
    game_df = game_df.sort_values(by=['at_bat_number', 'pitch_number'])
    all_batters = game_df['batter'].unique().tolist()
    #re-set the at_bat_number for the game to be sequential starting at 1
    at_bat_keys = game_df['at_bat_number'].unique().tolist()
    at_bat_values = range(1, len(at_bat_keys)+1)
    at_bat_map = dict(zip(at_bat_keys, at_bat_values))
    game_df['at_bat_number'] = game_df['at_bat_number'].replace(at_bat_map)
    
    #get the first 9 batter ids
    first_9_batter_subset = game_df[game_df['at_bat_number'] < 10]
    first_9_batters = first_9_batter_subset['batter'].unique().tolist()
    
    #map the batter id to batting order position 1-9
    batting_order_map = dict(zip(first_9_batters, range(1,10)))
    
    #for anyone else who bats later in the game, assign 'PH' (pinch hitter) to their batting order slot
    other_batters = list(set(all_batters) - set(first_9_batters))
    if len(other_batters) > 0:
        for batter in other_batters:
            batting_order_map[batter] = 'PH'
    try:
        game_df['batting_order_slot'] = game_df['batter'].apply(lambda x: batting_order_map[x])    
    except KeyError:
        game_df = None
        return game_df
    
    game_df['pitcher_AB'] = game_df['batter'].apply(lambda x: True if x in pitcher_list else False)
    game_df['batting_order_slot'] = game_df['batting_order_slot'].where(game_df['pitcher_AB'] == False, other='pitcher')
    return game_df

def get_pitch_tendencies(pitcher_df):
    #assign the normalized value counts for this pitchers pitch types to a dictionary
    pitcher_tendencies_overall = pitcher_df['pitch_cat'].value_counts(normalize=True).to_dict()

    #initialize empty dict for count categories tendencies
    pitcher_tendencies_by_count = {}
    
    #loop over each count category and get the pitchers tendencies and add to the dict
    for cat in pitcher_df['count_cat'].unique().tolist():
        subset = pitcher_df[pitcher_df['count_cat'] == cat]
        pitcher_tendencies_by_count[cat] = subset['pitch_cat'].value_counts(normalize=True).to_dict()
    return pitcher_tendencies_overall, pitcher_tendencies_by_count

def make_game_pitchcount_and_trailing_pitch_features_and_batting_order(pitcher_df, pitcher_list):
    df = pitcher_df.copy()
    all_games = []
    
    print('#pitches in df before: ' + str(len(df)))
    
    pitcher_tendencies_overall, pitcher_tendencies_by_count = get_pitch_tendencies(df)
    games = df['game_pk'].unique().tolist()
    
    for game in games:
        
        #take the first game and make the pitch count feature
        game_df = df[df['game_pk'] == game].copy()
        game_df['pitch_count'] = range(1, game_df.shape[0] + 1)
    
        #make the L1_pitch type feature:
        game_df['L1_pitch_type'] = game_df['pitch_cat'].shift(periods=1)
        game_df['L1_pitch_result'] = game_df['type'].shift(periods=1)
        game_df['L1_pitch_result'] = game_df['L1_pitch_result'].replace({np.nan:'first pitch'})
        game_df['L1_pitch_zone'] = game_df['zone'].shift(periods=1)
        game_df['L1_ball_high'] = game_df['ball_high'].shift(periods=1)
        game_df['L1_ball_low'] = game_df['ball_low'].shift(periods=1)
        game_df['L1_ball_left'] = game_df['ball_left'].shift(periods=1)
        game_df['L1_ball_right'] = game_df['ball_right'].shift(periods=1)
        
        game_df[['L1_pitch_zone', 'L1_ball_high', 'L1_ball_low', 'L1_ball_left', 'L1_ball_right']] = game_df[['L1_pitch_zone', 'L1_ball_high', 'L1_ball_low', 'L1_ball_left', 'L1_ball_right']].fillna(-1)
        #game_df['L1_pitch_zone'] = game_df['L1_pitch_zone'].fillna(-1)

        #overall strike % (to fill in for first 5 pitches L5_strike_perc)
        overall_strike_perc = df['type'].value_counts(normalize=True)['S'] * 100

        #make the trailing 5 pitches:
        for index, row in game_df.iterrows():
            #fill NaNs for L1_pitch using same method as when pitch_type was missing
            if row['pitch_count'] == 1:
                random_pitch = random.choices(population=list(pitcher_tendencies_overall.keys()), 
                                   weights=list(pitcher_tendencies_overall.values()), 
                                   k=1)[0]
                game_df.at[index, 'L1_pitch_type'] = random_pitch

            #for the first 5 rows, use overall pitcher tendencies    
            if row['pitch_count'] < 6:
                #fill with overall tendencies
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100
                    feature = 'L15_' + pitch + '_perc'
                    game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100
                #strike %
                game_df.at[index, 'L5_strike_perc'] = overall_strike_perc
                game_df.at[index, 'L15_strike_perc'] = overall_strike_perc

            else:
                current_pitch = game_df.at[index, 'pitch_count']
                #make a subset of the prev 5 pitches
                subset = game_df[(game_df['pitch_count'] > current_pitch - 6) & (game_df['pitch_count'] < current_pitch)]
                #grab the value count percentages for the last 5 pitches
                subset_percentages = subset['pitch_cat'].value_counts(normalize=True).to_dict()
                try:
                    L5_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                except KeyError:
                    L5_strike_perc = 0

                game_df.at[index, 'L5_strike_perc'] = L5_strike_perc

                #iterate over all possible pitch types this pitcher throws:
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    #if he has thrown that pitch type in last 5
                    try:
                        game_df.at[index, feature] = subset_percentages[pitch] * 100
                    #except for when he hasnt thrown that type in last 5
                    except:
                        game_df.at[index, feature] = 0


                if row['pitch_count'] < 16:
                    #make a subset of the prev 15 pitches
                    subset = game_df[(game_df['pitch_count'] < current_pitch)]
                    #grab the value count percentages for the last 15 pitches
                    subset_percentages = subset['pitch_cat'].value_counts(normalize=True).to_dict()
                    try:
                        L15_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                    except KeyError:
                        L15_strike_perc = 0

                    game_df.at[index, 'L15_strike_perc'] = L15_strike_perc

                    #iterate over all possible pitch types this pitcher throws:
                    for pitch in list(pitcher_tendencies_overall.keys()):
                        feature = 'L15_' + pitch + '_perc'
                        #if he has thrown that pitch type in last 15
                        try:
                            game_df.at[index, feature] = subset_percentages[pitch] * 100
                        #except for when he hasnt thrown that type in last 5
                        except:
                            game_df.at[index, feature] = 0
                else:
                    #make a subset of the prev 15 pitches
                    subset = game_df[(game_df['pitch_count'] > current_pitch - 16) & (game_df['pitch_count'] < current_pitch)]
                    #grab the value count percentages for the last 5 pitches
                    subset_percentages = subset['pitch_cat'].value_counts(normalize=True).to_dict()
                    try:
                        L15_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                    except KeyError:
                        L15_strike_perc = 0

                    game_df.at[index, 'L15_strike_perc'] = L15_strike_perc

                    #iterate over all possible pitch types this pitcher throws:
                    for pitch in list(pitcher_tendencies_overall.keys()):
                        feature = 'L15_' + pitch + '_perc'
                        #if he has thrown that pitch type in last 5
                        try:
                            game_df.at[index, feature] = subset_percentages[pitch] * 100
                        #except for when he hasnt thrown that type in last 5
                        except:
                            game_df.at[index, feature] = 0
                            
        #apply the battting order features to the game:                
        game_df = make_game_batting_order(game_df)
    
        all_games.append(game_df)
    
    new_df = pd.concat(all_games).sort_values(by=['game_date', 'game_pk', 'at_bat_number', 'pitch_number'])
    
    print('# pitches in df after: ' + str(len(new_df)))
    
    return new_df

def make_prev_ab_walk_basehit_run_and_homerun_features(pitcher_df):
    
    all_games = []
    #iterate over each game
    for game in pitcher_df['game_pk'].unique():
        #make subset df for that game
        game_df = pitcher_df[pitcher_df['game_pk'] == game].copy()
        #initialize columns to False:
        game_df['prev_ab_run_scored'] = False
        game_df['prev_ab_homerun'] = False
        game_df['prev_ab_walk'] = False
        game_df['prev_ab_basehit'] = False
        game_df['prev_ab_strikeout'] = False
        
        #this gets the 
        at_bats = game_df['at_bat_number'].sort_values().unique()
        
        #initialize empty dicts
        run_scored = []
        homeruns = []
        walks = []
        basehits = []
        strikeouts = []
        
        walks = ['walk', 'hit_by_pitch']
        basehits = ['single', 'double', 'triple', 'home_run']
        
        #starting w/ 2nd AB, iterate thru to the end of the at_bats:
        for ab in at_bats[2:]:
            #get the index for the last pitch of the prev AB
            prev_ab_last_pitch_index = game_df[game_df['at_bat_number'] == ab-1]['pitch_number'].index.max()
            #check if the last pitch resulted in a walk or hit by pitch:
            if game_df.loc[prev_ab_last_pitch_index]['events'] in walks:
                #if so, add an entry
                walks.append(ab)
            #check if last pitch gave up a basehit:
            elif game_df.loc[prev_ab_last_pitch_index]['events'] in basehits:
                basehits.append(ab)
            elif game_df.loc[prev_ab_last_pitch_index]['events'] == 'strikeout':
                strikeouts.append(ab)
            
            #to check if prev AB resulted in a run scoring: compare score before and after the AB
            prev_score = game_df[game_df['at_bat_number'] == ab-1]['bat_score'].values[0]
            current_score = game_df[game_df['at_bat_number'] == ab]['bat_score'].values[0]
            
            if current_score > prev_score:
                run_scored.append(ab)
                 
                #check if last AB gave up a homerun:
                if game_df.loc[prev_ab_last_pitch_index]['events'] == 'home_run':
                    homeruns.append(ab)
                    
        #iterate over each at_bat, and add the features to the df where appropriate
        for ab in at_bats:
            idx = game_df[game_df['at_bat_number'] == ab].index
            if ab in walks:
                game_df.at[idx, 'prev_ab_walk'] = True
            elif ab in basehits:
                game_df.at[idx, 'prev_ab_basehit'] = True
            elif ab in strikeouts:
                game_df.at[idx, 'prev_ab_strikeout'] = True
            if ab in run_scored:
                game_df.at[idx, 'prev_ab_run_scored'] = True
                if ab in homeruns:
                    game_df.at[idx, 'prev_ab_homerun'] = True
        all_games.append(game_df)
        
    return pd.concat(all_games).sort_values(by=['game_date', 'game_pk', 'pitch_count'])

In [21]:
%%time
pitcher_df = make_game_pitchcount_and_trailing_pitch_features_and_batting_order(pitcher_df, pitcher_list)

#pitches in df before: 6188
# pitches in df after: 6188
CPU times: user 1min 3s, sys: 11 ms, total: 1min 3s
Wall time: 1min 3s


In [22]:
%%time
pitcher_df = make_prev_ab_walk_basehit_run_and_homerun_features(pitcher_df)

CPU times: user 16.7 s, sys: 19.5 ms, total: 16.7 s
Wall time: 16.7 s


In [0]:
#pitcher_df[['game_pk','at_bat_number','bat_score','events', 'prev_ab_run_scored', 'prev_ab_homerun', 'prev_ab_walk', 'prev_ab_basehit', 'prev_ab_strikeout']].head(10)

In [24]:
cols = ['pitch_count', 'pitch_type', 'type', 'batting_order_slot']	

for col in pitcher_df.columns:
    if col[:2] == 'L1':
        cols.append(col)
    if col[-4:] == 'perc':
        if col[:2]=='L5':
            cols.append(col)
        if col[:3]=='L15':
            cols.append(col)
        
pitcher_df[cols].head(20)

,pitch_count,pitch_type,type,batting_order_slot,L1_pitch_type,L1_pitch_result,L1_pitch_zone,L1_ball_high,L1_ball_low,L1_ball_left,L1_ball_right,L5_fastball_perc,L15_fastball_perc,L15_fastball_perc,L5_breaking_perc,L15_breaking_perc,L15_breaking_perc,L5_offspeed_perc,L15_offspeed_perc,L15_offspeed_perc,L5_strike_perc,L15_strike_perc,L15_strike_perc
1510,1,FF,S,1,breaking,first pitch,-1.0,-1,-1,-1,-1,56.415643,56.415643,56.415643,40.982547,40.982547,40.982547,2.60181,2.60181,2.60181,54.250162,54.250162,54.250162
1511,2,FF,B,1,fastball,S,8.0,False,False,False,False,56.415643,56.415643,56.415643,40.982547,40.982547,40.982547,2.60181,2.60181,2.60181,54.250162,54.250162,54.250162
1512,3,SL,B,1,fastball,B,14.0,False,True,False,False,56.415643,56.415643,56.415643,40.982547,40.982547,40.982547,2.60181,2.60181,2.60181,54.250162,54.250162,54.250162
1513,4,FF,X,1,breaking,B,14.0,False,True,False,True,56.415643,56.415643,56.415643,40.982547,40.982547,40.982547,2.60181,2.60181,2.60181,54.250162,54.250162,54.250162
1514,5,FF,X,2,fastball,X,5.0,False,False,False,False,56.415643,56.415643,56.415643,40.982547,40.982547,40.982547,2.60181,2.60181,2.60181,54.250162,54.250162,54.250162
1515,6,FF,S,3,fastball,X,1.0,False,False,True,False,80.000000,80.000000,80.000000,20.000000,20.000000,20.000000,0.00000,0.00000,0.00000,20.000000,20.000000,20.000000
1516,7,FF,X,3,fastball,S,3.0,False,False,False,False,80.000000,83.333333,83.333333,20.000000,16.666667,16.666667,0.00000,0.00000,0.00000,20.000000,33.333333,33.333333
1531,8,FF,S,4,fastball,X,4.0,False,False,True,False,80.000000,85.714286,85.714286,20.000000,14.285714,14.285714,0.00000,0.00000,0.00000,20.000000,28.571429,28.571429
1532,9,CU,S,4,fastball,S,8.0,False,True,False,False,100.000000,87.500000,87.500000,0.000000,12.500000,12.500000,0.00000,0.00000,0.00000,40.000000,37.500000,37.500000
1533,10,SL,B,4,breaking,S,6.0,False,False,False,False,80.000000,77.777778,77.777778,20.000000,22.222222,22.222222,0.00000,0.00000,0.00000,60.000000,44.444444,44.444444


In [25]:
pitcher_df[cols].tail(20)

,pitch_count,pitch_type,type,batting_order_slot,L1_pitch_type,L1_pitch_result,L1_pitch_zone,L1_ball_high,L1_ball_low,L1_ball_left,L1_ball_right,L5_fastball_perc,L15_fastball_perc,L15_fastball_perc,L5_breaking_perc,L15_breaking_perc,L15_breaking_perc,L5_offspeed_perc,L15_offspeed_perc,L15_offspeed_perc,L5_strike_perc,L15_strike_perc,L15_strike_perc
1305642,68,FF,S,8,fastball,S,5.0,False,False,False,False,40.0,26.666667,26.666667,60.0,73.333333,73.333333,0.0,0.000000,0.000000,60.0,60.000000,60.000000
1305643,69,SL,B,8,fastball,S,3.0,False,False,False,False,60.0,26.666667,26.666667,40.0,73.333333,73.333333,0.0,0.000000,0.000000,60.0,60.000000,60.000000
1305644,70,CU,X,8,breaking,B,14.0,False,False,False,True,40.0,26.666667,26.666667,60.0,73.333333,73.333333,0.0,0.000000,0.000000,40.0,53.333333,53.333333
1305645,71,SL,B,9,breaking,X,9.0,False,True,False,False,40.0,26.666667,26.666667,60.0,73.333333,73.333333,0.0,0.000000,0.000000,40.0,46.666667,46.666667
1305646,72,SL,S,9,breaking,B,14.0,False,True,False,True,40.0,26.666667,26.666667,60.0,73.333333,73.333333,0.0,0.000000,0.000000,40.0,46.666667,46.666667
1305647,73,FF,X,9,breaking,S,9.0,False,False,False,False,20.0,20.000000,20.000000,80.0,80.000000,80.000000,0.0,0.000000,0.000000,40.0,46.666667,46.666667
1305648,74,FF,S,1,fastball,X,3.0,False,False,False,False,20.0,26.666667,26.666667,80.0,73.333333,73.333333,0.0,0.000000,0.000000,20.0,46.666667,46.666667
1305649,75,SL,B,1,fastball,S,4.0,False,False,False,False,40.0,33.333333,33.333333,60.0,66.666667,66.666667,0.0,0.000000,0.000000,40.0,53.333333,53.333333
1305650,76,SL,S,1,breaking,B,14.0,False,True,False,True,40.0,33.333333,33.333333,60.0,66.666667,66.666667,0.0,0.000000,0.000000,40.0,46.666667,46.666667
1305651,77,SL,S,1,breaking,S,6.0,False,False,False,False,40.0,33.333333,33.333333,60.0,66.666667,66.666667,0.0,0.000000,0.000000,60.0,46.666667,46.666667


## Fill Batting Scouting Report NaNs

#### Load in the batting_order_slot map pickle

In [0]:
batting_order_slot_map = pickle.load(open("/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/batting_order_slot_map.pkl", "rb"))

In [0]:
#batting_order_slot_map

In [0]:
batter_cols = ['fastball_perc_faced','fastball_chase_perc','fastball_bip_swung_perc', 'fastball_taken_strike_perc',
               'fastball_est_woba', 'fastball_babip', 'fastball_iso_value', 'breaking_perc_faced', 'breaking_chase_perc',
               'breaking_bip_swung_perc', 'breaking_taken_strike_perc', 'breaking_est_woba', 'breaking_babip', 
               'breaking_iso_value', 'offspeed_perc_faced', 'offspeed_chase_perc', 'offspeed_bip_swung_perc',
               'offspeed_taken_strike_perc', 'offspeed_est_woba', 'offspeed_babip', 'offspeed_iso_value',
               'pitchout_perc_faced']

def fill_batting_nans(pitcher_df, batting_order_slot_map):
    df = pitcher_df.copy()
    for slot in df['batting_order_slot'].unique().tolist():
        subset = df[df['batting_order_slot'] == slot].copy()
        df = df.drop(subset.index)
        for col in batter_cols:
            subset[col] = subset[col].fillna(batting_order_slot_map[slot][col])
        df = pd.concat([df, subset])
        print('finished w/ slot: ' + str(slot))
    df = df.sort_values(by=['game_date', 'game_pk', 'pitch_count'])
    return df

In [29]:
%%time
pitcher_df = fill_batting_nans(pitcher_df, batting_order_slot_map)

finished w/ slot: 1
finished w/ slot: 2
finished w/ slot: 3
finished w/ slot: 4
finished w/ slot: 5
finished w/ slot: 6
finished w/ slot: 7
finished w/ slot: 8
finished w/ slot: 9
finished w/ slot: pitcher
finished w/ slot: PH
CPU times: user 493 ms, sys: 3.97 ms, total: 497 ms
Wall time: 499 ms


## Add Pitcher / Batter Matchup History Report

In [0]:
def add_pb_matchup_priors(pitcher_df, pitcher_df17, start_dates, end_dates):
    df = pd.concat([pitcher_df, pitcher_df17], sort=False)
    
    #initialize empty list to store dfs (concat them together later)
    df_list = []
    
    #iterate over each period
    for i in range(len(start_dates)):
        
        #make the prior and current dfs:
        prior_df = df[df['game_date'] < start_dates[i]]
        current_df = df[(df['game_date'] >= start_dates[i]) & (df['game_date'] <= end_dates[i])]
        
        #get all the pitch_types this pitcher has thrown in the past:
        pitch_types = prior_df['pitch_cat'].unique().tolist()
        
        try:
            pitch_types.remove('PO')
        except:
            pass
        
        #get a list of the batters in the current_df
        current_batters = current_df['batter'].unique().tolist()
        
        batters_dict = {}
        
        current_df_list = []
        
        for batter in current_batters:
            batter_df_list = []
            
            #first use subset from prior df
            batter_subset = prior_df[prior_df['batter'] == batter].copy()
            #if pitcher has never faced this batter before:
            if batter_subset.empty:
                #get the left or right handedness of the batter
                stand = current_df[current_df['batter'] == batter]['stand'].values[0]
                #use overall prior tendencies vs left or right handed hitters
                overall, by_count = get_pitch_tendencies(prior_df[prior_df['stand'] == stand])
            else:
                overall, by_count = get_pitch_tendencies(batter_subset)
            batters_dict[batter] = by_count
            
            #now use subset of current_df where batter=batter
            batter_subset = current_df[current_df['batter'] == batter].copy()
            #iterate over the different count_cat types:
            for count_cat in ['ahead', 'behind', 'neutral']:
                count_subset = batter_subset[batter_subset['count_cat'] == count_cat].copy()
                if count_subset.empty:
                    continue
                else:
                    for pitch in pitch_types:
                        try:
                            count_subset['PB_'+pitch] = batters_dict[batter][count_cat][pitch] * 100
                        except KeyError:
                            count_subset['PB_'+pitch] = 0
                            
                current_df_list.append(count_subset)
        
        current_df = pd.concat(current_df_list, sort=False)
        df_list.append(current_df)
                    
    new_df = pd.concat(df_list, sort=False).sort_values(by=['game_date', 'game_pk', 'pitch_count'])
    return new_df

In [31]:
%%time
pitcher_df = add_pb_matchup_priors(pitcher_df, pitcher_df17, start_dates, end_dates)

CPU times: user 24.1 s, sys: 28.6 ms, total: 24.1 s
Wall time: 24.1 s


In [32]:
pitcher_df17.pitch_type.value_counts()

FF    2371
SL     876
CU     614
CH     161
FC      31
FT      12
Name: pitch_type, dtype: int64

In [33]:
pitcher_df.pitch_type.value_counts()

FF    3470
SL    1527
CU    1009
CH     161
FC      21
Name: pitch_type, dtype: int64

In [34]:
pitcher_df17.pitch_cat.value_counts(normalize=True)

fastball    0.593850
breaking    0.366544
offspeed    0.039606
Name: pitch_cat, dtype: float64

In [35]:
pitcher_df.pitch_cat.value_counts(normalize=True)

fastball    0.564156
breaking    0.409825
offspeed    0.026018
Name: pitch_cat, dtype: float64

In [0]:
#test[['pb_FF', 'pb_CH','pb_CU','pb_SL','pb_FT','pb_FC']]

## Choose features to use in model input vector, re-cast datatypes, encode categorical variables (onehot and binary), scale appropriately (Standard vs MinMax vs Robust), and check for any remaining NaNs and choose imputation strategy

#### Train/test Split (IMO use April/2017 - end of June/2019 as Train, and July/August 2019 as Test)

In [37]:
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_fastball_perc,count_cat_fastball_perc,overall_breaking_perc,count_cat_breaking_perc,overall_offspeed_perc,count_cat_offspeed_perc,pitch_count,L1_pitch_type,L1_pitch_result,L1_pitch_zone,L1_ball_high,L1_ball_low,L1_ball_left,L1_ball_right,L5_fastball_perc,L15_fastball_perc,L5_breaking_perc,L15_breaking_perc,L5_offspeed_perc,L15_offspeed_perc,L5_strike_perc,L15_strike_perc,batting_order_slot,pitcher_AB,prev_ab_run_scored,prev_ab_homerun,prev_ab_walk,prev_ab_basehit,prev_ab_strikeout,PB_fastball,PB_breaking,PB_offspeed
1510,FF,2018-03-29,22730,94.599998,-1.9486,6.8034,592261.0,434378.0,8.0,NaN,2018.0,-0.8339,1.6529,0.2748,1.7070,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,7.4453,-136.895203,-10.8637,-11.885600,32.316799,-9.809500,3.3339,1.4562,NaN,NaN,NaN,92.489998,2652.0,5.498,529411.0,55.000801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,S,NaN,Bot,180329_195119,Standard,1.0,NaN,False,False,False,False,0,0,1,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,0,Justin Verlander,NaN,called_strike,NaN,4-Seam Fastball,Standard,00,neutral,fastball,59.546127,69.760900,38.973853,30.098453,1.48002,0.140647,1.0,breaking,first pitch,-1.0,-1,-1,-1,-1,56.415643,56.415643,40.982547,40.982547,2.60181,2.60181,54.250162,54.250162,1,False,False,False,False,False,False,71.428571,28.571429,0.000000
1511,FF,2018-03-29,22711,94.300003,-1.9916,6.6522,592261.0,434378.0,14.0,NaN,2018.0,-1.2892,1.4338,0.4047,1.2020,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.9916,-136.566605,-11.4425,-18.015499,31.964199,-12.041500,3.3446,1.4668,NaN,NaN,NaN,93.283997,2638.0,6.095,529411.0,54.403400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195137,Standard,1.0,NaN,False,True,False,False,0,0,0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,1,Justin Verlander,NaN,ball,NaN,4-Seam Fastball,Standard,01,ahead,fastball,59.546127,50.609185,38.973853,47.141518,1.48002,2.249297,2.0,fastball,S,8.0,False,False,False,False,56.415643,56.415643,40.982547,40.982547,2.60181,2.60181,54.250162,54.250162,1,False,False,False,False,False,False,27.777778,72.222222,0.000000
1512,SL,2018-03-29,22699,86.400002,-2.1580,6.5734,592261.0,434378.0,14.0,NaN,2018.0,0.2707,0.3526,1.5138,-0.0429,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.0583,-125.145203,-10.0136,1.18

In [0]:
model_features = ['game_date', 'batter', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up',
                'inning','fielder_2', 'sz_top', 'sz_bot', 'game_pk','at_bat_number',
                'pitch_number', 'bat_score', 'fld_score', 'stand', 'p_throws',
                'inning_topbot', 'if_fielding_alignment','score_diff', 'on_1b_id',
                'L1_ball_high', 'L1_ball_low', 'L1_ball_left', 'L1_ball_right',
                'bases_loaded', 'fastball_perc_faced', 'fastball_chase_perc',
                'fastball_bip_swung_perc', 'fastball_taken_strike_perc',
                'fastball_est_woba', 'fastball_babip', 'fastball_iso_value',
                'breaking_perc_faced', 'breaking_chase_perc', 'breaking_bip_swung_perc',
                'breaking_taken_strike_perc', 'breaking_est_woba', 'breaking_babip',
                'breaking_iso_value', 'offspeed_perc_faced', 'offspeed_chase_perc',
                'offspeed_bip_swung_perc', 'offspeed_taken_strike_perc',
                'offspeed_est_woba', 'offspeed_babip', 'offspeed_iso_value',
                'pitchout_perc_faced', 'balls','strikes', 'of_fielding_alignment',
                '_count', 'count_cat', 'overall_breaking_perc',
                'count_cat_breaking_perc', 'overall_fastball_perc',
                'count_cat_fastball_perc', 'overall_offspeed_perc',
                'count_cat_offspeed_perc', 'pitch_count', 'L1_pitch_type',
                'L1_pitch_result', 'L1_pitch_zone', 'L5_fastball_perc', 
                'L15_fastball_perc', 'L5_breaking_perc', 'L15_breaking_perc',
                'L5_offspeed_perc', 'L15_offspeed_perc', 'L5_strike_perc',
                'L15_strike_perc', 'batting_order_slot', 'pitcher_AB', 
                'prev_ab_run_scored', 'prev_ab_homerun', 'prev_ab_walk',
                'prev_ab_basehit', 'prev_ab_strikeout', 'PB_fastball', 'PB_breaking',
                'PB_offspeed', 'pitch_cat']

def extract_date_features(df):
    df = df.copy()
    
    #df['day'] = df['game_date'].dt.day
    df['month'] = df['game_date'].dt.month
    df['year'] = df['game_date'].dt.year
    
    df = df.sort_values(by=['game_date', 'game_pk', 'pitch_count'])
    df = df.drop('game_date', axis=1)
    return df

In [0]:
model_df = pitcher_df[model_features].copy()

#rename fielder_2 to catcher_id:
model_df = model_df.rename(columns={'fielder_2':'catcher_id'})

#fillna for on_1b_id:
model_df['on_1b_id'] = model_df['on_1b_id'].fillna(-1)

#fillna for if and of alignment:
model_df[['if_fielding_alignment', 'of_fielding_alignment']] = model_df[['if_fielding_alignment', 'of_fielding_alignment']].fillna('Standard')

#fillna for sz_top and sz_bot
model_df['sz_top'] = model_df['sz_top'].fillna(model_df['sz_top'].mean())
model_df['sz_bot'] = model_df['sz_bot'].fillna(model_df['sz_bot'].mean())

#fillna for catcher_id
catcher = model_df['catcher_id'].value_counts().index[0]
model_df['catcher_id'] = model_df['catcher_id'].fillna(catcher)

#convert datetime into just month and year
model_df = extract_date_features(model_df)

#bool cols replace False and True with 0/1 (and some -1)
bool_cols =  ['L1_ball_high', 'L1_ball_low', 'L1_ball_left', 'L1_ball_right', 
              'bases_loaded', 'pitcher_AB', 'prev_ab_run_scored', 'prev_ab_homerun', 
              'prev_ab_walk', 'prev_ab_basehit', 'prev_ab_strikeout']

model_df[bool_cols] = model_df[bool_cols] * 1

In [60]:
model_df.head(1)

,batter,on_3b,on_2b,on_1b,outs_when_up,inning,catcher_id,sz_top,sz_bot,game_pk,at_bat_number,pitch_number,bat_score,fld_score,stand,p_throws,inning_topbot,if_fielding_alignment,score_diff,on_1b_id,L1_ball_high,L1_ball_low,L1_ball_left,L1_ball_right,bases_loaded,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,of_fielding_alignment,_count,count_cat,overall_breaking_perc,count_cat_breaking_perc,overall_fastball_perc,count_cat_fastball_perc,overall_offspeed_perc,count_cat_offspeed_perc,pitch_count,L1_pitch_type,L1_pitch_result,L1_pitch_zone,L5_fastball_perc,L15_fastball_perc,L5_breaking_perc,L15_breaking_perc,L5_offspeed_perc,L15_offspeed_perc,L5_strike_perc,L15_strike_perc,batting_order_slot,pitcher_AB,prev_ab_run_scored,prev_ab_homerun,prev_ab_walk,prev_ab_basehit,prev_ab_strikeout,PB_fastball,PB_breaking,PB_offspeed,pitch_cat,month,year
1510,592261.0,0.0,0.0,0.0,0.0,1.0,435263.0,3.3339,1.4562,529411.0,1.0,1.0,0.0,1.0,R,R,Bot,Standard,1.0,-1.0,-1,-1,-1,-1,0,66.241684,21.138212,42.050209,40.0,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.2,0.054545,6.76275,36.231884,21.666666,33.962265,0.426231,0.25,0.041667,0.0,0,0,Standard,00,neutral,38.973853,30.098453,59.546127,69.7609,1.48002,0.140647,1.0,breaking,first pitch,-1.0,56.415643,56.415643,40.982547,40.982547,2.60181,2.60181,54.250162,54.250162,1,0,0,0,0,0,0,71.428571,28.571429,0.0,fastball,3,2018


In [61]:
model_df.isna().sum()

batter                        0
on_3b                         0
on_2b                         0
on_1b                         0
outs_when_up                  0
inning                        0
catcher_id                    0
sz_top                        0
sz_bot                        0
game_pk                       0
at_bat_number                 0
pitch_number                  0
bat_score                     0
fld_score                     0
stand                         0
p_throws                      0
inning_topbot                 0
if_fielding_alignment         0
score_diff                    0
on_1b_id                      0
L1_ball_high                  0
L1_ball_low                   0
L1_ball_left                  0
L1_ball_right                 0
bases_loaded                  0
fastball_perc_faced           0
fastball_chase_perc           0
fastball_bip_swung_perc       0
fastball_taken_strike_perc    0
fastball_est_woba             0
fastball_babip                0
fastball

In [0]:
import category_encoders as ce
from sklearn.preprocessing import RobustScaler, StandardScaler

#categoricals cols:


#one hot encode categorical columns:
one_hot_cols = ['outs_when_up', 'stand', 'inning_topbot', 'if_fielding_alignment',
                'of_fielding_alignment', 'count_cat', 'L1_pitch_type', 
                'L1_pitch_result', 'L1_ball_high', 'L1_ball_low', 'L1_ball_left', 
                'L1_ball_right','year', 'p_throws']

#binary encode categorical columns:
binary_cols = ['inning', 'batter', 'catcher_id', 'game_pk', 'at_bat_number', 'pitch_number',
               'bat_score', 'fld_score', 'score_diff', 'on_1b_id', 'balls', 'strikes', 
               '_count', 'pitch_count', 'L1_pitch_zone', 'batting_order_slot', 
               'month']
    

#robust scaler cols:

numeric_cols = ['fastball_perc_faced', 'fastball_chase_perc', 'fastball_bip_swung_perc', 
                'fastball_taken_strike_perc', 'fastball_est_woba', 'fastball_babip', 
                'fastball_iso_value', 'breaking_perc_faced', 'breaking_chase_perc', 
                'breaking_bip_swung_perc', 'breaking_taken_strike_perc', 
                'breaking_est_woba', 'breaking_babip', 'breaking_iso_value', 
                'offspeed_perc_faced', 'offspeed_chase_perc', 'offspeed_bip_swung_perc', 
                'offspeed_taken_strike_perc','offspeed_est_woba', 'offspeed_babip', 
                'offspeed_iso_value','overall_breaking_perc', 'count_cat_breaking_perc', 
                'overall_fastball_perc', 'count_cat_fastball_perc', 'overall_offspeed_perc', 
                'count_cat_offspeed_perc', 'pitchout_perc_faced', 'L5_fastball_perc', 
                'L15_fastball_perc', 'L5_breaking_perc', 'L15_breaking_perc',
                'L5_offspeed_perc', 'L15_offspeed_perc', 'L5_strike_perc', 
                'L15_strike_perc', 'PB_fastball', 'PB_breaking', 'PB_offspeed', 
                'sz_bot', 'sz_top']

def final_model_preprocess(model_df):
    df = model_df.copy()
    
    #one hot encode certain columns
    one_hot_ce = ce.OneHotEncoder(cols=one_hot_cols, drop_invariant=False, 
                                  return_df=True, verbose=10, use_cat_names=True)
    
    one_hot_df = df[one_hot_cols].copy() 
    one_hot_df = one_hot_ce.fit_transform(one_hot_df)
    
    #binary encode columns w/ lots of categories
    binary_ce = ce.BinaryEncoder(cols=binary_cols, return_df=True)
    
    binary_df = df[binary_cols].copy()
    binary_df = binary_ce.fit_transform(binary_df)
    
    #grab the remaining features
    other_cols = set(df.columns) - set(one_hot_cols + binary_cols)
    other_df = df[other_cols].copy()
    
    #merge everything back together
    df = pd.concat([one_hot_df, binary_df, other_df], axis=1)
    return df

In [63]:
%%time
model_df = final_model_preprocess(model_df)

CPU times: user 554 ms, sys: 22.1 ms, total: 576 ms
Wall time: 583 ms


In [64]:
model_df.shape

(6188, 184)

In [65]:
model_df.head()

,outs_when_up_0.0,outs_when_up_1.0,outs_when_up_2.0,stand_L,stand_R,inning_topbot_Bot,inning_topbot_Top,if_fielding_alignment_Infield shift,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_Standard,of_fielding_alignment_4th outfielder,of_fielding_alignment_Strategic,count_cat_neutral,count_cat_ahead,count_cat_behind,L1_pitch_type_breaking,L1_pitch_type_fastball,L1_pitch_type_offspeed,L1_pitch_result_first pitch,L1_pitch_result_S,L1_pitch_result_B,L1_pitch_result_X,L1_ball_high_-1.0,L1_ball_high_0.0,L1_ball_high_1.0,L1_ball_low_-1.0,L1_ball_low_0.0,L1_ball_low_1.0,L1_ball_left_-1.0,L1_ball_left_0.0,L1_ball_left_1.0,L1_ball_right_-1.0,L1_ball_right_0.0,L1_ball_right_1.0,year_2018.0,year_2019.0,p_throws_L,p_throws_R,inning_0,inning_1,inning_2,inning_3,inning_4,batter_0,batter_1,batter_2,batter_3,batter_4,batter_5,batter_6,batter_7,batter_8,batter_9,catcher_id_0,catcher_id_1,catcher_id_2,game_pk_0,game_pk_1,game_pk_2,game_pk_3,game_pk_4,game_pk_5,game_pk_6,at_bat_number_0,at_bat_number_1,at_bat_number_2,at_bat_number_3,at_bat_number_4,at_bat_number_5,at_bat_number_6,pitch_number_0,pitch_number_1,pitch_number_2,pitch_number_3,pitch_number_4,bat_score_0,bat_score_1,bat_score_2,bat_score_3,fld_score_0,fld_score_1,fld_score_2,fld_score_3,fld_score_4,score_diff_0,score_diff_1,score_diff_2,score_diff_3,score_diff_4,on_1b_id_0,on_1b_id_1,on_1b_id_2,on_1b_id_3,on_1b_id_4,on_1b_id_5,on_1b_id_6,on_1b_id_7,on_1b_id_8,balls_0,balls_1,balls_2,strikes_0,strikes_1,strikes_2,_count_0,_count_1,_count_2,_count_3,_count_4,pitch_count_0,pitch_count_1,pitch_count_2,pitch_count_3,pitch_count_4,pitch_count_5,pitch_count_6,pitch_count_7,L1_pitch_zone_0,L1_pitch_zone_1,L1_pitch_zone_2,L1_pitch_zone_3,L1_pitch_zone_4,batting_order_slot_0,batting_order_slot_1,batting_order_slot_2,batting_order_slot_3,batting_order_slot_4,month_0,month_1,month_2,month_3,breaking_perc_faced,breaking_est_woba,fastball_est_woba,breaking_iso_value,offspeed_taken_strike_perc,prev_ab_homerun,offspeed_bip_swung_perc,offspeed_babip,offspeed_iso_value,pitcher_AB,count_cat_breaking_perc,L5_breaking_perc,breaking_taken_strike_perc,overall_breaking_perc,count_cat_offspeed_perc,fastball_perc_faced,L15_fastball_perc,overall_fastball_perc,pitch_cat,overall_offspeed_perc,prev_ab_basehit,bases_loaded,L5_strike_perc,PB_breaking,pitchout_perc_faced,breaking_chase_perc,fastball_iso_value,sz_top,sz_bot,offspeed_chase_perc,PB_fastball,PB_offspeed,L15_breaking_perc,prev_ab_strikeout,breaking_babip,count_cat_fastball_perc,on_1b,prev_ab_run_scored,fastball_bip_swung_perc,fastball_babip,L15_strike_perc,L15_offspeed_perc,offspeed_perc_faced,fastball_taken_strike_perc,fastball_chase_perc,breaking_bip_swung_perc,on_3b,L5_fastball_perc,prev_ab_walk,offspeed_est_woba,L5_offspeed_perc,on_2b
1510,1,0,0,0,1,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,26.995565,0.231478,0.300159,0.054545,33.962265,0,21.666666,0.250000,0.041667,0,30.098453,40.982547,43.915344,38.973853,0.140647,66.241684,56.415643,59.546127,fastball,1.48002,0,0,54.250162,28.571429,0.000000,27.516779,0.098684,3.3339,1.4562,36.231884,71.428571,0.000000,40.982547,0,0.200000,69.760900,0.0,0,42.050209,0.220395,54.250162,2.60181,6.762750,40.000000,21.138212,36.702129,0.0,56.415643,0,0.426231,2.60181,0.0
1511,1,0,0,0,1,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,26.995565,0.231478,0.300159,0.054545,33.962265,0,21.666666,0.250000,0.041667,0,47.141518,40.982547,43.915344,38.973853,2.249297,66.241684,56.415643,59.546127,fastball,1.48002,0,0,54.250162,72.222222,0.000000,27.516779,0.098684,3.3446,1.4668,36.2

In [0]:
#Todo:

# - pickle the model_df and export

# - scale the data (probably w/ robust scaler)

# - split into train and test

# - train models and evaluate predictions

In [0]:
model_df.to_pickle(path=("verlander_df.pkl"),compression='zip')

In [48]:
!ls

RC-v1.1--Data-Cleaning-and-Feature-Engineering	sample_data  verlander_df.pkl
